# Agent Architecture Components

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
my_api_key = os.getenv("ANTHROPIC_API_KEY")

In [7]:
from anthropic import Anthropic

client = Anthropic(
    api_key=my_api_key
)

## 1. Query Router

In [ ]:
# Agent architecture
class ResearchAgent:
    def __init__(self, llm, retriever, tools=None):
        self.llm = llm
        self.retriever = retriever  # Your existing ChromaDB retriever
        self.tools = tools or {}
        self.query_router = QueryRouter(llm)
    
    def process_query(self, query):
        # Main entry point for processing queries
        query_type = self.query_router.classify_query(query)
        
        if query_type == "direct_knowledge":
            return self.direct_response(query)
        elif query_type == "research_needed":
            return self.research_response(query)
        elif query_type == "tool_required":
            return self.tool_response(query)
    
    def direct_response(self, query):
        # Generate response directly from LLM
        # ...
    
    def research_response(self, query):
        # Get documents from retriever
        docs = self.retriever.get_relevant_documents(query)
        # Generate response with context
        # ...
    
    def tool_response(self, query):
        # Identify required tool
        tool_name = self.query_router.identify_tool(query)
        tool = self.tools.get(tool_name)
        # Execute tool and generate response
        # ...

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

class QueryRouter:
    def __init__(self, llm: ChatOpenAI, prompt_path: str):
        self.llm = llm
        self.prompt_template = PromptTemplate.from_file(
            prompt_path,
            input_variables=["KNOWLEDGE_BASE", "AVAILABLE_FUNCTIONS", "USER_QUERY"]
        )
    
    def classify_query(self, query: str, knowledge_base: str ="", availble_functions: str = "")-> str:
        """
        Use the LLM to classify the query based on the provided prompt.
        Returns one of: 'direct_knowledge', 'research_needed', or 'tool_required'.
        """
        filled_prompt = self.prompt_template.format(
            KNOWLEDGE_BASE=knowledge_base,
            AVAILABLE_FUNCTIONS=available_functions,
            USER_QUERY=query
        )
        response = self.llm([ HumanMessage(content=filled_prompt)]).content.lower()
        if "function call" in response:
            return "tool_required"
        elif "need for more context" in response:
            return "research_needed"
        elif "direct answer" in response:
            return "direct_knowledge"
        else:
            # Fallback to research if unclear
            return "research_needed"

        
    # def identify_tool(self, query):
    #     # Determine which tool is needed
    #     # ...

In [4]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

router = QueryRouter(llm=llm, prompt_path="query_classification_prompt_template.txt")

query = "What are the main differences between GPT-3.5 and GPT-4?"
classification = router.classify_query(query)

print("Query Type:", classification)


/tmp/ipykernel_11044/3461779143.py:8: DeprecationWarning: `input_variables' is deprecated and ignored.
  self.prompt_template = PromptTemplate.from_file(


NameError: name 'available_functions' is not defined

In [1]:
import dspy

class QueryClassifier(dspy.Signature):
    """Classify a user query as requiring direct knowledge, research, or tools."""
    query = dspy.InputField()
    query_type = dspy.OutputField(desc="One of: direct_knowledge, research_needed, tool_required")
    reasoning = dspy.OutputField(desc="Explanation for the classification")

class DirectResponseGenerator(dspy.Signature):
    """Generate a response using only the model's knowledge."""
    query = dspy.InputField()
    response = dspy.OutputField()

# Additional signature classes for other components

In [ ]:
class QueryClassifierModule(dspy.Module):
    def __init__(self):
        self.classifier = dspy.Predict(QueryClassifier)
    
    def forward(self, query):
        return self.classifier(query=query)

# Additional modules for other components

## 2. Direct Knowledge Responder

## 3. RAG-Enhanced Responder

## 4. Tool Caller

## 5. Multi-Step Reasoning Coordinator